In [ ]:
import matplotlib.pyplot as plt

from astropy.table import Table

%matplotlib notebook

In [ ]:
# Turn off interactive plotting by default.
# Call plt.show() to display an interactive plot
# Then if you want to create a new interactive plot, call
#
# plt.close()
# <...new plotting commands...>
# plt.show()
#
# This will result in the desired behavior that you can run the entire
# notebook and generate each figure as an interactive figure.
# Otherwise, if you just kept one figure interactive all plots would go to that figure.

plt.ioff()

In [ ]:
field = 'LSQ13cwp'
# field = 'PTF11mty'

In [ ]:
J_cat = Table.read('%s_%s_cat.fits' % (field, 'J'))
H_cat = Table.read('%s_%s_cat.fits' % (field, 'H'))
ref_table = Table.read('%s_ref_table.fits' % field)

snr_threshold = 5
good_color = (J_cat['J_SNR'] > snr_threshold) & (H_cat['H_SNR'] > snr_threshold)

J_cat = J_cat[good_color]
H_cat = H_cat[good_color]
ref_table = ref_table[good_color]

In [ ]:
def make_ds9_region_file(cat, galaxies, out_file='ra_dec.reg'):
    header = """# coord_ra, coord_dec
global point=circle
fk5
"""
    lines = [header]
    for row, gal in zip(cat, galaxies):
        line = "point({ra:0.8f}, {dec:0.8f})".format(
            ra=np.rad2deg(row['coord_ra']), dec=np.rad2deg(row['coord_dec']))
        if gal:
            line += " # point=box"
            
        line += "\n"
        lines.append(line) 
    
    with open(out_file, mode='w') as out:
        out.writelines(lines)

In [ ]:
extendedName = "base_ClassificationExtendedness_value"

In [ ]:
plt.hist(ref_table[extendedName], bins=np.linspace(-0.05, 1.05, 12), range=(-0.5, 1.5))
plt.xlabel('ClassificationExtendness')
plt.show()

In [ ]:
extendedness_threshold = 0.95  # [2018-04-04]  It's actually currently just 0 or 1.
stars = ref_table[extendedName] > extendedness_threshold
gal = ref_table[extendedName] < extendedness_threshold
# There are also NaNs

In [ ]:
make_ds9_region_file(H_cat, gal)

In [ ]:
cat = H_cat
plt.scatter(cat[stars]["slot_PsfShape_xx"], cat[stars]["slot_PsfShape_yy"], label='Stars')
plt.scatter(cat[gal]["slot_PsfShape_xx"], cat[gal]["slot_PsfShape_yy"], label='Galaxies')
plt.xlabel('PsfShape xx')
plt.ylabel('PsfShape yy')
plt.plot([0, 25], [0, 25], linestyle='--', color='grey')
plt.xlim(0, 25)
plt.ylim(0, 20)
plt.legend()

plt.show()

In [ ]:
plt.scatter(J_cat[stars]['J_mag']-H_cat[stars]['H_mag'], J_cat[stars]['J_mag'], label='Stars')
plt.scatter(J_cat[gal]['J_mag']-H_cat[gal]['H_mag'], J_cat[gal]['J_mag'], marker='+', label='Galaxies')
plt.xlabel('J-H [AB mag]')
plt.ylabel('J [AB mag]')
plt.ylim(24, 13)
plt.legend()
plt.show()

In [ ]:
plt.close()
plt.scatter(J_cat[stars]['J_mag'], H_cat[stars]['H_mag'], label='Stars')
plt.scatter(J_cat[gal]['J_mag'], H_cat[gal]['H_mag'], marker='+', label='Galaxies')
plt.xlabel('J [AB mag]')
plt.ylabel('H [AB mag]')
plt.legend()
plt.show()

In [ ]:
plt.close()
plt.scatter(J_cat[stars]['J_mag'], J_cat[stars]['J_mag_err'], label='J stars', color='blue')
plt.scatter(J_cat[gal]['J_mag'], J_cat[gal]['J_mag_err'], label='J galaxies', marker='+', color='blue')
plt.scatter(H_cat[stars]['H_mag'], H_cat[stars]['H_mag_err'], label='H stars', color='green')
plt.scatter(H_cat[gal]['H_mag'], H_cat[gal]['H_mag_err'], label='H galaxies', marker='+', color='green')

plt.xlabel('AB mag')
plt.ylabel('mag uncertainty')
plt.legend()
plt.show()